Домашнее задание 5

Напишите свою функцию определения качества модели по следующей метрике: максимальный precision, при условии, что $precision &lt; 1.5 * recall$ и $recall &gt; 0.5$ и определите наилучшее её значение, перебирая гиперпараметры по предложенной сетке
Ответом на эту задачу является максимальное значение качества по предложенной метрике, округлённое до 4го знак

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer

In [7]:
param_grid = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_depth': [None, 5, 10, 15, 20],
    'criterion': ['entropy', 'gini']
}

X_data, y_data = load_breast_cancer(return_X_y=True)

In [8]:
estimator = RandomForestClassifier(random_state=42)

print('Accuracy best params and score')
result = GridSearchCV(estimator, param_grid, cv=3, scoring='accuracy').fit(X_data, y_data)
print('\tParams:', result.best_params_)
print('\tScore:', result.best_score_)

Accuracy best params and score
	Params: {'criterion': 'entropy', 'max_depth': None, 'n_estimators': 50}
	Score: 0.9648506151142355


In [28]:
from sklearn.metrics import precision_recall_curve

def scorer_f(y_true, probas_pred, **args):
    precision, recall, thresholds = precision_recall_curve(y_true, probas_pred[:, 1])
    precisions = []
    for i in range(precision.size):
        if ((precision[i] < 1.5 * recall[i]) and (recall[i] > 0.5)):
            precisions.append(precision[i])
    return max(precisions)
scorer = make_scorer(scorer_f, greater_is_better=True, needs_proba=True)

print('Custom loss best params and score')
result = GridSearchCV(estimator, param_grid, cv=3, scoring=scorer).fit(X_data, y_data)
print('\tParams:', result.best_params_)
print('\tScore:', round(result.best_score_, 4))

Custom loss best params and score
	Params: {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 50}
	Score: 0.9942
